# **Laboratorio 2 - Regresión Logistica**
***Univ. Villca Jaita Lino Fernando - Ing. Cs. de la Computación***

En este ejercicio se implementa regresion logistica a un dataset que contiene un conjunto de datos sobre la satisfaccion de los pasajeros durante un vuelo en aerolineas considerando las siguientes caracteristicas:


> **Datos de entrada** como: id,género,tipo de cliente,edad,tipo de viaje,clase,distancia de vuelo,servicio wifi a bordo,hora de salida/llegada conveniente,facilidad de reserva en línea,ubicación de la puerta de embarque,comida y bebida,embarque en línea,comodidad del asiento,entretenimiento a bordo, Servicio a bordo, Servicio de sala de piernas, Manejo de equipaje, Servicio de check-in, Servicio a bordo, Limpieza, Retraso de salida en minutos, Retraso de llegada en minutos.


> **Dato de salida  (y a predecir):** **Satisfacción** (Satisfecho = 1 ) y (Neutral o insatisfecho = 0)



> Nombre del Dataset: Airline Passenger Satisfaction
*   URL del Dataset: https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction
*   Temática (lo que se quiere buscar con el dataset): Satisfaccion de los clientes con una Aerolinea
*   Formato: csv

**Consideraciones:** El dataset contiene dos archivos uno de entrenamiento y otro de test. utilizaremos solo el archivo de entrenamiento tanto para entrenar el modelo y realizar las pruebas como se plantea en el problema (80% y 20%).



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# se utiliza para el manejo de rutas y directorios.
import os

# Calculo cientifico y vectorial para python
import numpy as np

# Librerias para graficar
from matplotlib import pyplot

# Modulo de optimización de scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

## 1 Cargamos los datos desde nuestro Google Drive

In [8]:
#Importamos la libreria pandas para poder manipular nuestros datos que son literales y poder cambiarlos a numericos
import pandas as pd

# Cargar el dataset
data = pd.read_csv('/content/drive/MyDrive/SIS 420 INTELIGENCIA ARTIFICAL 2024/MIS DATASETS/LAB 2 DATASETS/airline passenger satisfaction/train.csv', delimiter=',')

# Verificar el tipo de datos
print(data.dtypes)

Unnamed: 0                             int64
id                                     int64
Gender                                object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Inflight wifi service                  int64
Departure/Arrival time convenient      int64
Ease of Online booking                 int64
Gate location                          int64
Food and drink                         int64
Online boarding                        int64
Seat comfort                           int64
Inflight entertainment                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Inflight service                       int64
Cleanliness                            int64
Departure 

In [11]:
# Convertir columnas categóricas a numéricas (astype('category').cat.codes, convierte las categorías en valores numéricos únicos.)
# Luego, cat.codes extrae estos códigos numéricos asignados a cada categoría.

data['Gender'] = data['Gender'].astype('category').cat.codes
data['Customer Type'] = data['Customer Type'].astype('category').cat.codes
data['Type of Travel'] = data['Type of Travel'].astype('category').cat.codes
data['Class'] = data['Class'].astype('category').cat.codes
data['satisfaction'] = data['satisfaction'].astype('category').cat.codes

# Verificar que las columnas categóricas se han convertido a numéricas
print(data.head())
print(data.dtypes)

   Unnamed: 0      id  Gender  Customer Type  Age  Type of Travel  Class  \
0           0   70172       1              0   13               1      2   
1           1    5047       1              1   25               0      0   
2           2  110028       0              0   26               0      0   
3           3   24026       0              0   25               0      0   
4           4  119299       1              0   61               0      0   

   Flight Distance  Inflight wifi service  Departure/Arrival time convenient  \
0              460                      3                                  4   
1              235                      3                                  2   
2             1142                      2                                  2   
3              562                      2                                  5   
4              214                      3                                  3   

   ...  Inflight entertainment  On-board service  Leg room ser

In [12]:
data.count()

Unnamed: 0                           103904
id                                   103904
Gender                               103904
Customer Type                        103904
Age                                  103904
Type of Travel                       103904
Class                                103904
Flight Distance                      103904
Inflight wifi service                103904
Departure/Arrival time convenient    103904
Ease of Online booking               103904
Gate location                        103904
Food and drink                       103904
Online boarding                      103904
Seat comfort                         103904
Inflight entertainment               103904
On-board service                     103904
Leg room service                     103904
Baggage handling                     103904
Checkin service                      103904
Inflight service                     103904
Cleanliness                          103904
Departure Delay in Minutes      

In [15]:
# al ver el conteo de filas o de ejemplos del dataset, se puede observar que la columna
# Arrival Delay in Minutes no tiene el mismo numero de datos porque tiene datos vacios
# procederemos a eliminar estas filas para evitar errores.
# Eliminar filas con valores faltantes en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Verificar las dimensiones después de eliminar las filas con valores faltantes
print(data.shape)
data.count()

(103594, 25)


Unnamed: 0                           103594
id                                   103594
Gender                               103594
Customer Type                        103594
Age                                  103594
Type of Travel                       103594
Class                                103594
Flight Distance                      103594
Inflight wifi service                103594
Departure/Arrival time convenient    103594
Ease of Online booking               103594
Gate location                        103594
Food and drink                       103594
Online boarding                      103594
Seat comfort                         103594
Inflight entertainment               103594
On-board service                     103594
Leg room service                     103594
Baggage handling                     103594
Checkin service                      103594
Inflight service                     103594
Cleanliness                          103594
Departure Delay in Minutes      

Ahora nuestros ejemplos estan completos y contienes valores consistentes en cada una de sus columnas. Teniendo en total: 103594 ejemplos.
De los cuales utilizaremos (82875 ejemplos (80%) para entrenamiento) y (20719 ejemplos (20%) para el test).

In [17]:
# Realizamos la asignacion de datos correspondiente a cada dataset
X_train, y_train = data.iloc[:82875, 1:23], data.iloc[:82875, 23]
X_test, y_test = data.iloc[82875:, 1:23], data.iloc[82875:, 23]
X = X_train
y = y_train
print(X)
print(y)

           id  Gender  Customer Type  Age  Type of Travel  Class  \
0       70172       1              0   13               1      2   
1        5047       1              1   25               0      0   
2      110028       0              0   26               0      0   
3       24026       0              0   25               0      0   
4      119299       1              0   61               0      0   
...       ...     ...            ...  ...             ...    ...   
83115  105101       0              0   45               0      0   
83116   82823       0              1   28               0      0   
83117   42651       0              0   18               0      2   
83118  100217       0              0   12               1      1   
83119   43092       1              0   70               1      1   

       Flight Distance  Inflight wifi service  \
0                  460                      3   
1                  235                      3   
2                 1142              

In [21]:
def sigmoid(z):
    # Calcula la sigmoide de una entrada z
    # convierte la intrada a un arreglo numpy
    z = np.array(z)

    g = np.zeros(z.shape)

    g = 1 / (1 + np.exp(-z))

    return g

Se calcula el valor de la sigmoide aplicando la funcion sigmoid con `z=0`, se debe obtener un resultado de 0.5. RE recomienda experimentar con otros valores de `z`.

In [22]:
# Prueba la implementacion de la funcion sigmoid
z = [0, 0.5, 1]
g = sigmoid(z)

print('g(', z, ') = ', g)

g( [0, 0.5, 1] ) =  [0.5        0.62245933 0.73105858]


In [23]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [24]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

<a id="section2"></a>
#### 1.2.2 Función de Costo y Gradiente

Se implementa la funcion cost y gradient, para la regresión logistica. Antes de continuar es importante agregar el termino de intercepcion a X.

In [25]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

La funcion de costo en una regresión logistica es:

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \left[ -y^{(i)} \log\left(h_\theta\left( x^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - h_\theta\left( x^{(i)} \right) \right) \right]$$

y el gradiente del costo es un vector de la misma longitud como $\theta$ donde el elemento $j^{th}$ (para $j = 0, 1, \cdots , n$) se define como:

$$ \frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} $$

Si bien este gradiente parece idéntico al gradiente de regresión lineal, la fórmula es diferente porque la regresión lineal y logística tienen diferentes definiciones de $h_\theta(x)$.
<a id="costFunction"></a>

In [26]:
def calcularCosto(theta, X, y):
    # Inicializar algunos valores utiles
    m = y.size  # numero de ejemplos de entrenamiento

    J = 0
    h = sigmoid(X.dot(theta.T))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))

    return J

In [27]:
def descensoGradiente(theta, X, y, alpha, num_iters):
    # Inicializa algunos valores
    m = y.shape[0] # numero de ejemplos de entrenamiento

    # realiza una copia de theta, el cual será acutalizada por el descenso por el gradiente
    theta = theta.copy()
    J_history = []

    for i in range(num_iters):
        h = sigmoid(X.dot(theta.T))
        theta = theta - (alpha / m) * (h - y).dot(X)

        J_history.append(calcularCosto(theta, X, y))
    return theta, J_history

In [ ]:
# Elegir algun valor para alpha (probar varias alternativas)
alpha = 0.00001
num_iters = 50000

# inicializa theta y ejecuta el descenso por el gradiente
theta = np.zeros(23)
theta, J_history = descensoGradiente(theta, X, y, alpha, num_iters)

# Grafica la convergencia del costo
pyplot.plot(np.arange(len(J_history)), J_history, lw=2)
pyplot.xlabel('Numero de iteraciones')
pyplot.ylabel('Costo J')

# Muestra los resultados del descenso por el gradiente
print('theta calculado por el descenso por el gradiente: {:s}'.format(str(theta)))

In [48]:
print(X_test)
print(theta)

            id  Gender  Customer Type  Age  Type of Travel  Class  \
83120   110198       0              0    8               0      2   
83121     7747       0              1   25               0      0   
83122   115704       0              0   57               0      0   
83123    69899       0              1   24               0      0   
83124   117106       1              0   39               0      0   
...        ...     ...            ...  ...             ...    ...   
103899   94171       0              1   23               0      1   
103900   73097       1              0   49               0      0   
103901   68825       1              1   30               0      0   
103902   54173       0              1   22               0      1   
103903   62567       1              0   27               0      0   

        Flight Distance  Inflight wifi service  \
83120              1056                      3   
83121              1013                      2   
83122              15

In [50]:
def predecir(theta, X):
    # Calcula las predicciones utilizando la función sigmoide
    predictions = sigmoid(X.dot(theta))
    return predictions

# Normalizar las características del conjunto de prueba
X_test_norm, _, _ = featureNormalize(X_test)

# Agregar una columna de unos para el término de sesgo
m_test, _ = X_test_norm.shape
X_test_norm_with_bias = np.column_stack((np.ones(m_test), X_test_norm))

# Realizar predicciones en los datos de prueba
predictions_test = predecir(theta, X_test_norm_with_bias)

# Ajustar las predicciones a 0 o 1 usando un umbral
threshold = 0.5
predicted_labels_test = (predictions_test > threshold).astype(int)

# Calcular la precisión del modelo en los datos de prueba
accuracy_test = np.mean(predicted_labels_test == y_test) * 100

print('Precisión del modelo en los datos de prueba: {:.2f}%'.format(accuracy_test))




Precisión del modelo en los datos de prueba: 27.36%


In [ ]:
#Podemos ver las y_reales del dataset con las y_predichas por el modelo
# Realiza las predicciones en el conjunto de prueba
y_predichas = predecir(theta, X_test_norm)

# Ajusta las predicciones a valores binarios (0 o 1) utilizando un umbral
umbral = 0.5
y_predichas_binarias = (y_predichas > umbral).astype(int)

# Imprime las etiquetas reales y las etiquetas predichas
print("Etiquetas reales:")
print(y_test.values)  # Imprime las etiquetas reales del conjunto de prueba
print("\nEtiquetas predichas:")
print(y_predichas_binarias)  # Imprime las etiquetas predichas